In [ ]:
import regex as re
import nltk
from nltk.corpus import stopwords
import math
nltk.download('stopwords')



sentences = list()
sentence = list()
with open('annotations_hsy2.tsv') as file:
  for line in file:
      if line != "\n":
        sentence.append(line.split('\t')[1])
      if line == "\n" and sentence:
        sentences.append(sentence)
        sentence = list()

In [36]:
import re

stop_words = set(stopwords.words('english'))
clear_data = []

def clean_sentence(sentence):
    cleaned = []
    for lemma in sentence:
        cleaned_token = re.sub(r"(?<!\w)\.(?!\w)|[^\w\s@._-]+", "", lemma.lower())
        if cleaned_token and cleaned_token not in stop_words:
            cleaned.append(cleaned_token)
    return cleaned

for sentence in sentences:
    clear_data.append(clean_sentence(sentence))

In [41]:
trigrams = list()
for sentence in clear_data:
    for i in range(len(sentence) - 2):
        trigrams.append(sentence[i:i+3])
        
trigram_frequency = dict()
for trigram in trigrams:
    if tuple(trigram) not in trigram_frequency:
        # If not existed, create it
        trigram_frequency[tuple(trigram)] = 1
    else:
        # If existed, add it.
        trigram_frequency[tuple(trigram)] += 1


In [42]:
def mi_score(trigram_frequencies, word_counts, total_words):
    # Initialize a dictionary to store the MI scores for each trigram
    miscores = {}
    
    # Iterate over each trigram and its frequency in the trigram_frequencies dictionary
    for trigram, count in trigram_frequencies.items():
        # Calculate the denominator: product of the counts of individual words in the trigram
        denominator = word_counts[trigram[0]] * word_counts[trigram[1]] * word_counts[trigram[2]]
        
        # Calculate the mutual information (MI) using the provided formula
        mi = math.log2((total_words ** 2) * count / denominator)
        
        # Store the calculated MI score for the current trigram
        miscores[trigram] = mi
    
    # Return the dictionary containing the MI scores for all trigrams
    return miscores

In [43]:
from collections import Counter

words_count = Counter()

for sentence in clear_data:
    words_count.update(sentence)

total_words = sum(words_count.values())

mi_score = mi_score(trigram_frequency, words_count, total_words)

sorted(mi_score.items(), key=lambda x: x[1], reverse=True)[:30]

In [ ]:
import nltk
from nltk.collocations import TrigramAssocMeasures, TrigramCollocationFinder

nltk.download('punkt')

flat_text = [word for sentence in clear_data for word in sentence]

finder_thr = TrigramCollocationFinder.from_words(flat_text)

trigram_measures = TrigramAssocMeasures()
top_30_trigrams = finder_thr.nbest(trigram_measures.pmi, 30)

print(top_30_trigrams)